In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch

dataset = load_dataset("issai/kazqad")

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

def preprocess_function(examples):
    outputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=512,
        padding="max_length",
        return_offsets_mapping=True
    )

    start_positions = []
    end_positions = []

    print(f"Examples: {examples}")

    for i in range(len(examples['question'])):
        offset_mapping = outputs["offset_mapping"][i]
        input_ids = outputs["input_ids"][i]

        print(f"Example {i} Offset Mapping: {offset_mapping}")
        print(f"Example {i} Input IDs: {input_ids}")

        if 'answers' in examples:
            answer = examples['answers'][i]
            if 'text' in answer and 'answer_start' in answer:
                answer_texts = answer['text']
                answer_starts = answer['answer_start']

                print(f"Example {i} Answer Texts: {answer_texts}")
                print(f"Example {i} Answer Starts: {answer_starts}")

                if len(answer_texts) > 0 and len(answer_starts) > 0:
                    start_char = answer_starts[0]
                    end_char = start_char + len(answer_texts[0])

                    start_token_index = None
                    end_token_index = None

                    for idx, (start, end) in enumerate(offset_mapping):
                        if start <= start_char < end:
                            start_token_index = idx
                            break

                    for idx, (start, end) in enumerate(offset_mapping):
                        if start < end_char <= end:
                            end_token_index = idx
                            break

                    print(f"Processing example {i}:")
                    print("Answer Text:", answer_texts[0])
                    print("Start Char Index:", start_char)
                    print("End Char Index:", end_char)
                    print("Start Token Index:", start_token_index)
                    print("End Token Index:", end_token_index)

                    if start_token_index is None:
                        start_token_index = 0
                    if end_token_index is None or end_token_index >= len(input_ids):
                        end_token_index = len(input_ids) - 1

                    start_positions.append(start_token_index)
                    end_positions.append(end_token_index)
                else:
                    start_positions.append(0)
                    end_positions.append(0)
            else:
                start_positions.append(0)
                end_positions.append(0)
        else:
            start_positions.append(0)
            end_positions.append(0)

    outputs['start_positions'] = start_positions
    outputs['end_positions'] = end_positions

    outputs.pop("offset_mapping", None)
    return outputs

try:
    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
except Exception as e:
    print("Error during processing:", e)

In [14]:
tokenized_datasets['validation'][0]

{'input_ids': [101,
  538,
  28588,
  12670,
  24612,
  13237,
  21720,
  10993,
  16417,
  22840,
  64712,
  11571,
  10375,
  136,
  102,
  509,
  20411,
  25363,
  118,
  541,
  111085,
  71742,
  89306,
  542,
  41069,
  26205,
  28993,
  13257,
  570,
  66963,
  12095,
  88044,
  547,
  62289,
  30189,
  26367,
  36268,
  208,
  61861,
  33440,
  220,
  10234,
  10351,
  119,
  510,
  41069,
  26205,
  48097,
  10519,
  67501,
  605,
  40270,
  24519,
  10352,
  10375,
  15773,
  103116,
  10205,
  62025,
  87439,
  13237,
  557,
  10205,
  89094,
  57794,
  38074,
  108110,
  10987,
  105922,
  541,
  16417,
  15726,
  19913,
  10227,
  117,
  553,
  104808,
  13237,
  117,
  57684,
  11323,
  61947,
  31890,
  46195,
  570,
  66963,
  66013,
  28993,
  19079,
  37721,
  55655,
  11890,
  119,
  509,
  20411,
  25363,
  118,
  554,
  10292,
  208,
  38074,
  108110,
  57794,
  570,
  28588,
  12670,
  24612,
  13237,
  21384,
  40628,
  220,
  18609,
  547,
  10191,
  10352,
  10

In [15]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

True
1
Using device: cuda


In [16]:
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")
model.to(device)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000, 
    save_total_limit=2, 
    warmup_steps=500, 
    fp16=True,
    gradient_accumulation_steps=2, 
    report_to="none",
    dataloader_pin_memory=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

print(f"Model device: {next(model.parameters()).device}")

trainer.train()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0


  0%|          | 10/5270 [00:11<53:55,  1.63it/s] 

{'loss': 6.2985, 'grad_norm': 8.222498893737793, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.02}


  0%|          | 20/5270 [00:16<47:31,  1.84it/s]

...

100%|██████████| 5270/5270 [35:28<00:00,  2.64it/s]

{'loss': 0.1957, 'grad_norm': 0.601735532283783, 'learning_rate': 2.5157232704402518e-08, 'epoch': 9.99}



100%|██████████| 5270/5270 [35:41<00:00,  2.46it/s]

{'eval_loss': 6.327608108520508, 'eval_runtime': 13.205, 'eval_samples_per_second': 57.857, 'eval_steps_per_second': 19.311, 'epoch': 9.99}
{'train_runtime': 2141.5432, 'train_samples_per_second': 14.77, 'train_steps_per_second': 2.461, 'train_loss': 1.2860708624633486, 'epoch': 9.99}


TrainOutput(global_step=5270, training_loss=1.2860708624633486, metrics={'train_runtime': 2141.5432, 'train_samples_per_second': 14.77, 'train_steps_per_second': 2.461, 'total_flos': 8257500106371072.0, 'train_loss': 1.2860708624633486, 'epoch': 9.990521327014218})

In [17]:
results = trainer.evaluate()

print(results)

model.save_pretrained("./kazakh_qa_model")
tokenizer.save_pretrained("./kazakh_qa_model")

  0%|          | 0/255 [00:00<?, ?it/s]

100%|██████████| 255/255 [00:14<00:00, 17.61it/s]


{'eval_loss': 6.327608108520508, 'eval_runtime': 14.4973, 'eval_samples_per_second': 52.7, 'eval_steps_per_second': 17.59, 'epoch': 9.990521327014218}


('./kazakh_qa_model\\tokenizer_config.json',
 './kazakh_qa_model\\special_tokens_map.json',
 './kazakh_qa_model\\vocab.txt',
 './kazakh_qa_model\\added_tokens.json',
 './kazakh_qa_model\\tokenizer.json')

In [24]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

model_path = "./kazakh_qa_model(5e)"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

question = "Шоқан Уәлиханов қандай аймақтарда зерттеу жұмыстарын жүргізді?"
context = (
    "Шоқан Уәлиханов (1835–1865) — қазақтың ғалымы, тарихшысы, этнографы, географы, саяхатшысы, әрі саяси қайраткері. "
    "Ол қазақ халқының тарихын, мәдениетін және тұрмысын зерттеуде үлкен үлес қосты. Шоқан Уәлиханов Орыс Географиялық Қоғамының мүшесі болды "
    "және өзінің еңбектерімен танымал болды. Ол Жоңғария, Қашқария, Тянь-Шань аймақтарында зерттеу жұмыстарын жүргізді."
)

inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=512)

inputs = {key: val.to(device) for key, val in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

start_scores = outputs.start_logits
end_scores = outputs.end_logits

start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

if end_index < start_index:
    end_index = start_index

all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
answer_tokens = all_tokens[start_index:end_index+1]

answer_tokens = [token for token in answer_tokens if token not in tokenizer.all_special_tokens]

answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f"Question: {question}")
print(f"Context: {context}")
print(f"Start Index: {start_index}, End Index: {end_index}")
print(f"Answer Tokens: {answer_tokens}")
print(f"Answer: {answer}")


Question: Шоқан Уәлиханов қандай аймақтарда зерттеу жұмыстарын жүргізді?
Context: Шоқан Уәлиханов (1835–1865) — қазақтың ғалымы, тарихшысы, этнографы, географы, саяхатшысы, әрі саяси қайраткері. Ол қазақ халқының тарихын, мәдениетін және тұрмысын зерттеуде үлкен үлес қосты. Шоқан Уәлиханов Орыс Географиялық Қоғамының мүшесі болды және өзінің еңбектерімен танымал болды. Ол Жоңғария, Қашқария, Тянь-Шань аймақтарында зерттеу жұмыстарын жүргізді.
Start Index: 126, End Index: 141
Answer Tokens: ['Ж', '##о', '##ң', '##ға', '##рия', ',', 'Қ', '##аш', '##қа', '##рия', ',', 'Тя', '##нь', '-', 'Ш', '##ань']
Answer: Жоңғария, Қашқария, Тянь - Шань


In [19]:
dataset['validation'][713]

{'id': 'lit2252lit#31945_4_1',
 'title': 'Шалкиіз Тіленшіұлы',
 'context': 'Шалкиіз жырау (1465-1560) - ақын, жырау, батыр. Шалкиіз - туындылары орыс тіліне аударылған ақын-жыраулардың бірі. Оның шығармашылығын сол дәуірдегі орыс оқымыстылары мен зерттеушілері жоғары бағалаған. Темір биге айтқандары әртүрлі жинақтар мен басылымдарда бірнеше мәрте басылды. Жыраудың қазақ көне әдебиетіндегі орны ерекше. Шалкиіз жырау ғүмырының көп бөлігін Ноғай ордасының билеушісі Темір бидің қол астында өткізеді. Турашылдығымен, әділдігімен, даналығымен ел алдында беделді болады. Өмірінің соңғы жылдарын Хақназар хан тұсындағы Қазақ мемлекетінде өткізеді.',
 'question': 'Шалкиіз жырау қай биге толғау шығарған?',
 'answers': {'text': ['Темір биге'], 'answer_start': [203]}}